In [1]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

from nltk.tag.stanford import StanfordPOSTagger
from stanford_postagger.stanford_wrapper import StanfordPOSTagger as StanfordPOSTaggerWrapper

from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

import scipy
from sklearn.grid_search import RandomizedSearchCV

/home/agus/anaconda3/envs/basic/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/agus/anaconda3/envs/basic/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
%load_ext autoreload
%autoreload 2

# Read Dataset

In [3]:
f = open('datasets/conll2003/train.txt', 'r')
lines = f.readlines()
f.close()

del lines[0]
del lines[0]

dataset = []
sentence = []
for line in lines:
    splitter = line.strip().split(' ')
    if splitter[0] == '':
        continue
    elif (splitter[0] == '-DOCSTART-'):
        dataset.append(sentence)
        sentence = []
    else:
        token = splitter[0]
        tag = splitter[3]
        sentence.append((token, tag))

In [4]:
def convert_conlltxt2dataset(filename):
    f = open(filename, 'r')
    lines = f.readlines()
    f.close()
    
    del lines[0]
    del lines[0]
    
    dataset = []
    sentence = []
    for line in lines:
        splitter = line.strip().split(' ')
        if splitter[0] == '':
            continue
        elif (splitter[0] == '-DOCSTART-'):
            dataset.append(sentence)
            sentence = []
        else:
            token = splitter[0]
            tag = splitter[3]
            sentence.append((token, tag))
    return dataset

In [5]:
validation_dataset = convert_conlltxt2dataset('datasets/conll2003/valid.txt')
test_dataset = convert_conlltxt2dataset('datasets/conll2003/test.txt')

# Add Postag to Dataset

## Example

In [6]:
postagger = StanfordPOSTaggerWrapper()
postag = postagger.tag('+44 171')
postag

[('+44', 'CD'), ('171', 'CD')]

In [7]:
def add_postag2dataset(dataset):
    postagger = StanfordPOSTaggerWrapper()
    dataset_with_postag = []
    for sent in dataset:
        postagged_sent = []
        for index, (token, tag) in enumerate(sent):
            postagged_token = postagger.tag(token)
            postagged_sent.append((token, postagged_token[0][1], tag))
        dataset_with_postag.append(postagged_sent)
        
    return dataset_with_postag

postagged_validation_dataset = add_postag2dataset(validation_dataset)
postagged_test_dataset = add_postag2dataset(test_dataset)

# Delete Unused Dataset
del validation_dataset
del test_dataset

# Extract Feature

In [8]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    # Ortographic Feature, Word, POSTag & N-Gram
    features = {
        'word': word,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word[:2]': word[:2],
        'word[:3]': word[:3],
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'word.isupper()': word.isupper(),
        'postag': postag,
        'postag[:2]': postag[:2]
    }
    
    # Position
    features.update({
        'pos_front': i,
        'pos_end': len(sent) - i
    })
    
    # Bag Of Words
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
        
    if i < len(sent) - 1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2postag(sent):
    return [postag for token, postag, label in sent]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

# Feature Extraction

In [9]:
X_val = [sent2features(sent) for sent in postagged_validation_dataset]
y_val = [sent2labels(sent) for sent in postagged_validation_dataset]

X_test = [sent2features(sent) for sent in postagged_test_dataset]
y_test = [sent2labels(sent) for sent in postagged_test_dataset]

del postagged_validation_dataset
del postagged_test_dataset

# Load Word Embedding and Add to The Feature

In [10]:
def load_word_embedding_model(filename):
    f = open(filename, 'r', encoding='utf-8')
    lines = f.readlines()
    del lines[0]

    embedding_dict = {}
    counter = 0
    for line in lines:
        line = line.strip()
        arr = line.split(' ')
        word = arr[0]
        vectors = [float(x) for x in arr[1:]]
        embedding_dict[word] = vectors
    return embedding_dict

def add_word_embedding2feature(embedding_dict, feature):
    VECTOR_SPACE_SIZE = 50
    for i, sentence in enumerate(feature):
        for j, token in enumerate(sentence):
            word = token['word']
            vector = []
            if word in embedding_dict:
                vector = embedding_dict[word]
            else:
                vector = [0 for i in range(VECTOR_SPACE_SIZE)]
            for k in range(len(vector)):
                wk = 'w{}'.format(k+1)
                feature[i][j][wk] = vector[k]
    return feature

from copy import deepcopy

In [11]:
cbow_embedding_dict = load_word_embedding_model('models/word_embedding/cbow_model_2.txt')
X_cbow = deepcopy(X_test)
X_cbow = add_word_embedding2feature(cbow_embedding_dict, X_cbow)

X_val_cbow = deepcopy(X_val)
X_val_cbow = add_word_embedding2feature(cbow_embedding_dict, X_val_cbow)

del cbow_embedding_dict

In [12]:
sg_embedding_dict = load_word_embedding_model('models/word_embedding/sg_model_2.txt')
X_sg = deepcopy(X_test)
X_sg = add_word_embedding2feature(sg_embedding_dict, X_sg)

X_val_sg = deepcopy(X_val)
X_val_sg = add_word_embedding2feature(sg_embedding_dict, X_val_sg)

del sg_embedding_dict

In [13]:
fasttext_embedding_dict = load_word_embedding_model('models/word_embedding/fasttext_model_2.txt')
X_fasttext = deepcopy(X_test)
X_fasttext = add_word_embedding2feature(fasttext_embedding_dict, X_fasttext)

X_val_fasttext = deepcopy(X_val)
X_val_fasttext = add_word_embedding2feature(fasttext_embedding_dict, X_val_fasttext)

del fasttext_embedding_dict

In [14]:
del X_test

In [15]:
X_cbow[0][0]

{'word': 'SOCCER',
 'word.lower()': 'soccer',
 'word[-3:]': 'CER',
 'word[-2:]': 'ER',
 'word[:2]': 'SO',
 'word[:3]': 'SOC',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NN',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 419,
 'BOS': True,
 '+1:word.lower()': '-',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': ':',
 '+1:postag[:2]': ':',
 'w1': -1.4087068,
 'w2': 0.2011912,
 'w3': 0.99614793,
 'w4': -1.5684234,
 'w5': 0.63709104,
 'w6': -2.2249558,
 'w7': -0.30708793,
 'w8': -0.49136025,
 'w9': 1.1458724,
 'w10': -1.3158027,
 'w11': -0.9098756,
 'w12': 0.345007,
 'w13': -1.3596588,
 'w14': 1.544882,
 'w15': 2.7605484,
 'w16': 2.0274143,
 'w17': 3.2250345,
 'w18': -0.89328986,
 'w19': -2.1821504,
 'w20': 0.95720494,
 'w21': -1.113573,
 'w22': -0.68710643,
 'w23': 0.32429016,
 'w24': 1.9363227,
 'w25': -0.28782746,
 'w26': 1.6744254,
 'w27': 0.37658474,
 'w28': -2.6737828,
 'w29': 0.3630372,
 'w30': 1.5332483,
 'w3

In [16]:
X_sg[0][0]

{'word': 'SOCCER',
 'word.lower()': 'soccer',
 'word[-3:]': 'CER',
 'word[-2:]': 'ER',
 'word[:2]': 'SO',
 'word[:3]': 'SOC',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NN',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 419,
 'BOS': True,
 '+1:word.lower()': '-',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': ':',
 '+1:postag[:2]': ':',
 'w1': -0.5838318,
 'w2': -0.11893116,
 'w3': 0.42615455,
 'w4': -0.17929605,
 'w5': 0.6879327,
 'w6': -0.28123328,
 'w7': 0.10536976,
 'w8': -0.6712705,
 'w9': 0.8743664,
 'w10': -0.6807716,
 'w11': -1.2997047,
 'w12': 0.24734432,
 'w13': -0.43122563,
 'w14': 1.2878635,
 'w15': 1.4279467,
 'w16': 0.95324004,
 'w17': 1.1890935,
 'w18': -0.7155137,
 'w19': -0.8404136,
 'w20': 0.4906798,
 'w21': -0.9079721,
 'w22': -0.33419672,
 'w23': -0.24617764,
 'w24': 1.3376828,
 'w25': -0.49355692,
 'w26': 0.70783764,
 'w27': 1.2184808,
 'w28': -2.2577746,
 'w29': 0.71416575,
 'w30': 1.072876

In [17]:
X_fasttext[0][0]

{'word': 'SOCCER',
 'word.lower()': 'soccer',
 'word[-3:]': 'CER',
 'word[-2:]': 'ER',
 'word[:2]': 'SO',
 'word[:3]': 'SOC',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NN',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 419,
 'BOS': True,
 '+1:word.lower()': '-',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': ':',
 '+1:postag[:2]': ':',
 'w1': 0.34921017,
 'w2': 0.45019245,
 'w3': 0.7714098,
 'w4': 0.12115693,
 'w5': 0.48165482,
 'w6': -0.13365725,
 'w7': -0.50420403,
 'w8': -0.52196044,
 'w9': -0.21790357,
 'w10': 0.6755344,
 'w11': 0.24068084,
 'w12': 0.84111416,
 'w13': 0.43975905,
 'w14': -0.005092337,
 'w15': -0.3991359,
 'w16': -0.3375171,
 'w17': -0.263034,
 'w18': -0.8224232,
 'w19': -0.15123643,
 'w20': 0.18632635,
 'w21': -0.792666,
 'w22': 0.3734644,
 'w23': -0.098219275,
 'w24': 0.046064775,
 'w25': -0.29583707,
 'w26': -0.2676103,
 'w27': 0.021400599,
 'w28': -0.7760419,
 'w29': 0.16247961,
 'w30': 

# Load Model

In [18]:
import pickle

In [21]:
filename_cbow = 'temp_crf_cbow_2.sav'
loaded_model_cbow = pickle.load(open(filename_cbow, 'rb'))

filename_sg = 'temp_crf_sg_2.sav'
loaded_model_sg = pickle.load(open(filename_sg, 'rb'))

filename_fasttext = 'temp_crf_fasttext_2.sav'
loaded_model_fasttext = pickle.load(open(filename_fasttext, 'rb'))

In [22]:
labels = list(loaded_model_cbow.classes_)
labels.remove('O')
labels

['B-ORG', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG', 'I-MISC', 'I-LOC']

In [23]:
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# Cross Val Score

In [24]:
x_val_score_cbow = cross_val_score(loaded_model_cbow, X_val_cbow, y_val, cv=5, scoring=f1_scorer)
print(x_val_score_cbow)
print(x_val_score_cbow.mean())

[0.83587345 0.68975167 0.83332266 0.81602672 0.69750685]
0.7744962674906454


In [25]:
x_val_score_sg = cross_val_score(loaded_model_sg, X_val_sg, y_val, cv=5, scoring=f1_scorer)
print(x_val_score_sg)
print(x_val_score_sg.mean())

[0.81260701 0.69108331 0.83762899 0.8009353  0.69352107]
0.7671551361566453


In [26]:
x_val_score_fasttext = cross_val_score(loaded_model_fasttext, X_val_fasttext, y_val, cv=5, scoring=f1_scorer)
print(x_val_score_fasttext)
print(x_val_score_fasttext.mean())

[0.81677592 0.69550519 0.81810068 0.81043453 0.68456584]
0.7650764333524792


# CBOW

In [27]:
y_pred_cbow = loaded_model_cbow.predict(X_cbow)
metrics.flat_f1_score(y_test, y_pred_cbow,
                      average='weighted', labels=labels)

0.8103988855290419

# SG

In [28]:
y_pred_sg = loaded_model_sg.predict(X_sg)
metrics.flat_f1_score(y_test, y_pred_sg,
                      average='weighted', labels=labels)

0.8190264879565053

# FastText

In [29]:
y_pred_fasttext = loaded_model_fasttext.predict(X_fasttext)
metrics.flat_f1_score(y_test, y_pred_fasttext,
                      average='weighted', labels=labels)

0.812664066644544